In [1]:
import pandas as pd

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from concurrent.futures import ThreadPoolExecutor

from bs4 import BeautifulSoup as bs
import requests as req

import time

In [2]:
#Cookies 
options=Options()
options.add_argument(r'user-data-dir=C:\Users\matia\anaconda3\Lib\site-packages\selenium\cookies') #Cargamos las cookies

In [3]:
url = 'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query=abner'

PATH = 'driver/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(PATH, options=options)

driver.get(url)

In [5]:
html = driver.page_source

In [6]:
from bs4 import BeautifulSoup as bs

In [7]:
soup = bs(html, 'html.parser')  

In [8]:
soup.find('td', class_='rechts hauptlink').text

'7,00 mill. €'

In [9]:
stats2  = pd.read_csv('../../csv/csv_limpios/stats.csv')
lista_jugadores = stats2['Jugador'].tolist()
lista_jugadores = [nombre.replace(' ', '+') for nombre in lista_jugadores]
lista_jugadores

['Abner',
 'Álvaro+Aceves',
 'Marcos+Acuña',
 'Julen+Agirrezabala',
 'Álvaro+Aguado',
 'Joseph+Aidoo',
 'Sergio+Akieme',
 'Paul+Akouokou',
 'David+Alaba',
 'Tomás+Alarcón',
 'Jordi+Alba',
 'Raúl+Albiol',
 'Rubén+Alcaraz',
 'Omar+Alderete',
 'Iván+Alejo',
 'Carles+Aleñá',
 'Alejandro+Alfaro',
 'Ángel+Algobia',
 'Mohamed+Ali+Cho',
 'Domingos+André+Ribeiro+Almeida',
 'Marcos+Alonso',
 'Carlos+Álvarez',
 'Gastón+Álvarez',
 'Yeray+Álvarez',
 'Selim+Amallah',
 'Amath',
 'Jordan+Amavi',
 'Marcos+André',
 'Fabrizio+Angileri',
 'Josep+Antoni+Gayá',
 'José+Antonio',
 'Anuar',
 'Mauro+Arambarri',
 'Ronald+Araújo',
 'Adu+Ares',
 'Sergio+Arribas',
 'Roberto+Arroyo',
 'Ricard+Artero',
 'Santiago+Arzamendia',
 'Sergio+Asenjo',
 'Marco+Asensio',
 'Iago+Aspas',
 'Pierre-Emerick+Aubameyang',
 'Ludwig+Augustinsson',
 'Ezequiel+Ávila',
 'Iddrisu+Baba',
 'Loïc+Bade',
 'Édgar+Badía',
 'Alex+Baena',
 'Alejandro+Balde',
 'Mikel+Balenziaga',
 'Iván+Balliu',
 'Léo+Baptistão',
 'Keidi+Bare',
 'Kike+Barja',
 'And

In [10]:
def transfermarkt(x):
    try:
        driver = webdriver.Chrome(PATH, options=options)
        driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={x}')
        html = driver.page_source
        soup = bs(html, 'html.parser')
        return soup.find('td', class_='rechts hauptlink').text
    except:
        print(f'{x} no fue encontrado')
        return "0.00"

In [16]:
transfermarkt('abner')

'7,00 mill. €'

In [12]:
lista_jugadores

['Abner',
 'Álvaro+Aceves',
 'Marcos+Acuña',
 'Julen+Agirrezabala',
 'Álvaro+Aguado',
 'Joseph+Aidoo',
 'Sergio+Akieme',
 'Paul+Akouokou',
 'David+Alaba',
 'Tomás+Alarcón',
 'Jordi+Alba',
 'Raúl+Albiol',
 'Rubén+Alcaraz',
 'Omar+Alderete',
 'Iván+Alejo',
 'Carles+Aleñá',
 'Alejandro+Alfaro',
 'Ángel+Algobia',
 'Mohamed+Ali+Cho',
 'Domingos+André+Ribeiro+Almeida',
 'Marcos+Alonso',
 'Carlos+Álvarez',
 'Gastón+Álvarez',
 'Yeray+Álvarez',
 'Selim+Amallah',
 'Amath',
 'Jordan+Amavi',
 'Marcos+André',
 'Fabrizio+Angileri',
 'Josep+Antoni+Gayá',
 'José+Antonio',
 'Anuar',
 'Mauro+Arambarri',
 'Ronald+Araújo',
 'Adu+Ares',
 'Sergio+Arribas',
 'Roberto+Arroyo',
 'Ricard+Artero',
 'Santiago+Arzamendia',
 'Sergio+Asenjo',
 'Marco+Asensio',
 'Iago+Aspas',
 'Pierre-Emerick+Aubameyang',
 'Ludwig+Augustinsson',
 'Ezequiel+Ávila',
 'Iddrisu+Baba',
 'Loïc+Bade',
 'Édgar+Badía',
 'Alex+Baena',
 'Alejandro+Balde',
 'Mikel+Balenziaga',
 'Iván+Balliu',
 'Léo+Baptistão',
 'Keidi+Bare',
 'Kike+Barja',
 'And

In [13]:
lst3 = []

for e in lista_jugadores:
    if e not in lst3:
        lst3.append(e)

In [14]:
lst3

['Abner',
 'Álvaro+Aceves',
 'Marcos+Acuña',
 'Julen+Agirrezabala',
 'Álvaro+Aguado',
 'Joseph+Aidoo',
 'Sergio+Akieme',
 'Paul+Akouokou',
 'David+Alaba',
 'Tomás+Alarcón',
 'Jordi+Alba',
 'Raúl+Albiol',
 'Rubén+Alcaraz',
 'Omar+Alderete',
 'Iván+Alejo',
 'Carles+Aleñá',
 'Alejandro+Alfaro',
 'Ángel+Algobia',
 'Mohamed+Ali+Cho',
 'Domingos+André+Ribeiro+Almeida',
 'Marcos+Alonso',
 'Carlos+Álvarez',
 'Gastón+Álvarez',
 'Yeray+Álvarez',
 'Selim+Amallah',
 'Amath',
 'Jordan+Amavi',
 'Marcos+André',
 'Fabrizio+Angileri',
 'Josep+Antoni+Gayá',
 'José+Antonio',
 'Anuar',
 'Mauro+Arambarri',
 'Ronald+Araújo',
 'Adu+Ares',
 'Sergio+Arribas',
 'Roberto+Arroyo',
 'Ricard+Artero',
 'Santiago+Arzamendia',
 'Sergio+Asenjo',
 'Marco+Asensio',
 'Iago+Aspas',
 'Pierre-Emerick+Aubameyang',
 'Ludwig+Augustinsson',
 'Ezequiel+Ávila',
 'Iddrisu+Baba',
 'Loïc+Bade',
 'Édgar+Badía',
 'Alex+Baena',
 'Alejandro+Balde',
 'Mikel+Balenziaga',
 'Iván+Balliu',
 'Léo+Baptistão',
 'Keidi+Bare',
 'Kike+Barja',
 'And

In [15]:
len(lista_jugadores)

3340

In [17]:
# Hago un loop para que a cada jugador de lst3 le aplique la función transfermarkt y lo guarde en una lista
lst4 = []
for e in lst3:
    lst4.append(transfermarkt(e))
    time.sleep(0.5)

KeyboardInterrupt: 

In [ ]:
transfermarkt(lista_jugadores[0])

In [ ]:
#Scrapeo para sacar el valor de un jugador
driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text

In [ ]:
def valor(futbolista, path):
    try:
        driver = webdriver.Chrome(path)
        driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
        
        # Espera hasta que el elemento esté presente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink'))
        )
        
        try:
            valor = driver.find_element(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink').text
        except NoSuchElementException:
            valor = '0'
            print(f'No se encontró el valor de {futbolista}')

    except WebDriverException as e:
        valor = '0'
        print(f'Error con WebDriver al procesar {futbolista}: {e}')
    finally:
        driver.quit()

    return valor

In [ ]:
#Hago una funcion para scrapear el valor de todos mis jugadores
'''def valor(futbolista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    time.sleep(2)
    driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
    time.sleep(2)
    try:
        valor = driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text
    except:
        valor = '0'
        print(f'No se encontro el valor de {futbolista}')

    driver.quit()
    return valor
'''

In [ ]:
#Pruebo la funcion
valor('Carvajal',PATH)

# Se lo aplico a mi tabla completa

In [ ]:
stats2  = pd.read_csv('../../csv/csv_limpios/stats.csv')

In [ ]:
lista_jugadores = stats2['Jugador'].tolist()

In [ ]:
lista_jugadores

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [ ]:
stats2['Valor Mercado'] = resultados

In [ ]:
#Voy a buscar los jugadores que tienen valor 0 
stats2[stats2['Valor Mercado'] == '-']

# Lo aplico a temporada 2023


In [ ]:
jugadores_2023 = pd.read_csv('../../csv/csv_limpios/jugadores_2023_lp.csv')

In [ ]:
lista_jugadores_23 = jugadores_2023['Jugador'].tolist()

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_23]
    resultados_23 = [f.result() for f in concurrent.futures.as_completed(futures)]

In [ ]:
jugadores_2023['Valor Mercado'] = resultados_23

# Exporto a csv limpio

In [ ]:
stats2.to_csv('../../csv/csv_limpios/stats_completo.csv', index=False)
jugadores_2023.to_csv('../../csv/csv_limpios/jugadores_2023_lp.csv', index=False)